In [1]:
import os
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"
api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI


# BASE_URL = "https://api.zhizengzeng.com/v1"

# we use below approach to transfer Base_Url  to model and others
os.environ["OPENAI_API_BASE"] ="https://api.zhizengzeng.com/v1/"

llm = ChatOpenAI(model="gpt-4o-mini")

In [9]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, chain

contextualize_instructions = """Convert the latest user question into a standalone question given the chat history. Don't answer the question, return the question and nothing else (no descriptive text)."""
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_instructions),
        ("placeholder", "{chat_history}"),
        ("human", "{question}"),
    ]
)
contextualize_question = contextualize_prompt | llm | StrOutputParser()

qa_instructions = (
    """Answer the user question given the following context:\n\n{context}."""
)
qa_prompt = ChatPromptTemplate.from_messages(
    [("system", qa_instructions), ("human", "{question}")]
)


@chain
def contextualize_if_needed(input_: dict) -> Runnable:
    if input_.get("chat_history"):
        # NOTE: This is returning another Runnable, not an actual output.
        return contextualize_question
    else:
        return RunnablePassthrough() | itemgetter("question")


@chain
def fake_retriever(input_: dict) -> str:
    return "egypt's population in 2024 is about 111 million"


full_chain = (
    RunnablePassthrough.assign(question=contextualize_if_needed).assign(
        context=fake_retriever
    )
    | qa_prompt
    | llm
    | StrOutputParser()
)

full_chain.invoke(
    {
        "question": "what about egypt",
    }
)

"As of 2024, Egypt's population is estimated to be about 111 million. If you need more specific information about Egypt, such as its economy, culture, or geography, feel free to ask!"

#### check 

In [30]:
run_chain = RunnablePassthrough() | itemgetter("question")

In [31]:
print(type(run_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [32]:
run_chain.get_graph().print_ascii()

       +------------------+        
       | PassthroughInput |        
       +------------------+        
                 *                 
                 *                 
                 *                 
          +-------------+          
          | Passthrough |          
          +-------------+          
                 *                 
                 *                 
                 *                 
+--------------------------------+ 
| Lambda(itemgetter('question')) | 
+--------------------------------+ 
                 *                 
                 *                 
                 *                 
         +--------------+          
         | LambdaOutput |          
         +--------------+          



#### partial with functions

In [33]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 07/24/2024, 20:43:32


In [34]:
partial_prompt.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
+----------------------+ 
| PromptTemplateOutput | 
+----------------------+ 
